In [2]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, HTML, display_html
import seaborn as sns
import datetime as dt
import tensorflow as tf
import IPython
import holidays
from datetime import date
import datetime
import calendar
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from backports.datetime_fromisoformat import MonkeyPatch
from sklearn.metrics import mean_squared_error,mean_absolute_error

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from datetime import timezone, datetime, timedelta
# random search linear regression model on the auto insurance dataset
from scipy.stats import loguniform
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV

In [3]:
df = pd.read_csv('main_data/NSW.csv')
dfmx = pd.read_csv('main_data/NSW_MAX_TEMP/NSW_Data.csv')
dfmn = pd.read_csv('main_data/NSW_MIN_TEMP/NSW_Data.csv')
df_temp = pd.read_csv('weather.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,b'REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,0,1,nNSW1,2016/12/01 00:30:00,7100.66,52.97,TRADE
1,1,2,nNSW1,2016/12/01 01:00:00,6818.00,42.08,TRADE
2,2,3,nNSW1,2016/12/01 01:30:00,6538.58,50.16,TRADE
3,3,4,nNSW1,2016/12/01 02:00:00,6367.53,47.64,TRADE
4,4,5,nNSW1,2016/12/01 02:30:00,6213.00,45.50,TRADE


In [4]:
df_temp[['dt_iso']] = pd.to_datetime(df_temp['dt_iso'], cache=True, exact=True, format='%Y-%m-%d %H:%M:%S %z UTC', utc=True)
df_temp = df_temp.groupby(by='dt_iso').mean()
df_temp.index = df_temp.index.tz_convert('Australia/Sydney')
df_temp.index = df_temp.index.strftime('%Y-%m-%d %H:%M:%S')
df_temp.head()

,dt,timezone,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id
dt_iso,,,,,,,,,,,,,,,,,,,,
1990-01-01 11:00:00,631152000.0,39600.0,-33.86882,151.209295,27.51,27.99,20.92,32.0,1013.0,NaN,NaN,40.0,0.5,350.0,NaN,NaN,NaN,NaN,3.0,800.0
1990-01-01 12:00:00,631155600.0,39600.0,-33.86882,151.209295,27.84,27.69,20.79,32.0,1013.0,NaN,NaN,46.0,2.6,30.0,NaN,NaN,NaN,NaN,10.0,800.0
1990-01-01 13:00:00,631159200.0,39600.0,-33.86882,151.209295,27.82,27.78,20.78,31.8,1012.0,NaN,NaN,47.0,2.6,30.0,NaN,NaN,NaN,NaN,20.0,801.0
1990-01-01 14:00:00,631162800.0,39600.0,-33.86882,151.209295,27.50,24.96,21.03,31.0,1011.0,NaN,NaN,48.0,6.2,60.0,NaN,NaN,NaN,NaN,19.0,801.0
1990-01-01 15:00:00,631166400.0,39600.0,-33.86882,151.209295,27.56,23.64,21.35,31.0,1010.0,NaN,NaN,48.0,8.2,40.0,NaN,NaN,NaN,NaN,28.0,802.0


In [5]:
df = df[['SETTLEMENTDATE','TOTALDEMAND','RRP']]
df[['SETTLEMENTDATE']] = pd.to_datetime(df['SETTLEMENTDATE'], exact=True, cache=True, format='%Y-%m-%d %H:%M:%S')
df = df.sort_values(by='SETTLEMENTDATE')
df.head()

,SETTLEMENTDATE,TOTALDEMAND,RRP
201543,1999-01-01 00:30:00,6820.56000,27.64
201544,1999-01-01 01:00:00,6421.34833,26.64
201545,1999-01-01 01:30:00,5991.45000,25.81
201546,1999-01-01 02:00:00,5663.07333,19.94
201547,1999-01-01 02:30:00,5384.04667,19.48


In [6]:
# Group data by number of listings per date
df_example = df.groupby(by='SETTLEMENTDATE').mean()

# Change index to datetime
df_example.index = pd.to_datetime(df_example.index)

# Sort the values
df_example = df_example.sort_index(ascending = True)

df_example = df_example.tail(-1)

df_example = df_example.asfreq(freq='1H')

# Fill values with 0
df_example = df_example.fillna(value = 0)

df_example

,TOTALDEMAND,RRP
SETTLEMENTDATE,,
1999-01-01 01:00:00,6421.34833,26.64
1999-01-01 02:00:00,5663.07333,19.94
1999-01-01 03:00:00,5199.31500,15.13
1999-01-01 04:00:00,5008.23667,14.19
1999-01-01 05:00:00,5022.93000,12.41
...,...,...
2020-09-16 20:00:00,8394.08000,57.47
2020-09-16 21:00:00,7938.99000,55.13
2020-09-16 22:00:00,7541.11000,38.92


In [7]:
df_example = df_example.merge(df_temp, left_index=True, right_index=True, how='inner')
df_example = df_example.fillna(0)
df_example.head()

,TOTALDEMAND,RRP,dt,timezone,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id
1999-01-01 01:00:00,6421.34833,26.64,915112800.0,39600.0,-33.86882,151.209295,21.73,22.14,21.00,22.60,1020.0,0.0,0.0,85.0,4.1,90.0,0.0,0.0,0.0,0.0,75.0,803.0
1999-01-01 02:00:00,5663.07333,19.94,915116400.0,39600.0,-33.86882,151.209295,21.60,22.90,21.00,22.04,1019.0,0.0,0.0,88.0,3.1,110.0,0.0,0.0,0.0,0.0,75.0,803.0
1999-01-01 03:00:00,5199.31500,15.13,915120000.0,39600.0,-33.86882,151.209295,21.97,23.61,21.77,22.04,1018.0,0.0,0.0,94.0,3.6,120.0,0.0,0.0,0.0,0.0,75.0,803.0
1999-01-01 04:00:00,5008.23667,14.19,915123600.0,39600.0,-33.86882,151.209295,22.18,23.49,21.77,22.40,1018.0,0.0,0.0,89.0,3.6,120.0,0.0,0.0,0.0,0.0,75.0,803.0
1999-01-01 05:00:00,5022.93000,12.41,915127200.0,39600.0,-33.86882,151.209295,21.98,21.29,21.79,22.10,1018.0,0.0,0.0,88.0,6.2,90.0,0.0,0.0,0.0,0.0,75.0,803.0


In [8]:
# dfmx['date'] = pd.to_datetime(dfmx[['Year', 'Month', 'Day']])
# dfmx = dfmx.rename(columns = {'Maximum temperature (Degree C)': 'max_temp'})
# df_max = dfmx.groupby(by = 'date').agg({'max_temp': 'sum'})
# max_mask = (df_max.index.date >= df_example.index[0]) & (df_max.index.date <= df_example.index[len(df_example)-1])

# dfmn['date'] = pd.to_datetime(dfmn[['Year', 'Month', 'Day']])
# dfmn = dfmn.rename(columns = {'Minimum temperature (Degree C)': 'min_temp'})
# df_min = dfmn.groupby(by = 'date').agg({'min_temp': 'sum'})
# min_mask = (df_min.index.date >= df_example.index[0]) & (df_min.index.date <= df_example.index[len(df_example)-1])

# df_example['max_temp'] = df_max.loc[max_mask]
# df_example['min_temp'] = df_min.loc[min_mask]
# df_example['avg'] = df_example[['max_temp', 'min_temp']].mean(axis=1)

# Holiday added
aus_holidays = holidays.CountryHoliday('AUS', prov='NSW')
df_example['isHoliday'] = 0
for i,j in enumerate(df_example.index):    
    if j in aus_holidays:
        df_example['isHoliday'][i] = 1
    else:
        df_example['isHoliday'][i] = 0

df_example.head()

,TOTALDEMAND,RRP,dt,timezone,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,isHoliday
1999-01-01 01:00:00,6421.34833,26.64,915112800.0,39600.0,-33.86882,151.209295,21.73,22.14,21.00,22.60,1020.0,0.0,0.0,85.0,4.1,90.0,0.0,0.0,0.0,0.0,75.0,803.0,1
1999-01-01 02:00:00,5663.07333,19.94,915116400.0,39600.0,-33.86882,151.209295,21.60,22.90,21.00,22.04,1019.0,0.0,0.0,88.0,3.1,110.0,0.0,0.0,0.0,0.0,75.0,803.0,1
1999-01-01 03:00:00,5199.31500,15.13,915120000.0,39600.0,-33.86882,151.209295,21.97,23.61,21.77,22.04,1018.0,0.0,0.0,94.0,3.6,120.0,0.0,0.0,0.0,0.0,75.0,803.0,1
1999-01-01 04:00:00,5008.23667,14.19,915123600.0,39600.0,-33.86882,151.209295,22.18,23.49,21.77,22.40,1018.0,0.0,0.0,89.0,3.6,120.0,0.0,0.0,0.0,0.0,75.0,803.0,1
1999-01-01 05:00:00,5022.93000,12.41,915127200.0,39600.0,-33.86882,151.209295,21.98,21.29,21.79,22.10,1018.0,0.0,0.0,88.0,6.2,90.0,0.0,0.0,0.0,0.0,75.0,803.0,1


In [9]:
df_example['hour'] = df_example.index.hour
df_example['day'] = df_example.index.day
df_example['dayofweek'] = df_example.index.dayofweek
df_example['dayofyear'] = df_example.index.dayofyear
df_example['month'] = df_example.index.month
df_example['weekofyear'] = df_example.index.weekofyear

In [10]:
# df_example1.head()
# df_example2.head()
df_example

,TOTALDEMAND,RRP,dt,timezone,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,isHoliday,hour,day,dayofweek,dayofyear,month,weekofyear
1999-01-01 01:00:00,6421.34833,26.64,9.151128e+08,39600.0,-33.86882,151.209295,21.73,22.14,21.00,22.60,1020.0,0.0,0.0,85.0,4.10,90.0,0.0,0.0,0.0,0.0,75.0,803.0,1,1,1,4,1,1,53
1999-01-01 02:00:00,5663.07333,19.94,9.151164e+08,39600.0,-33.86882,151.209295,21.60,22.90,21.00,22.04,1019.0,0.0,0.0,88.0,3.10,110.0,0.0,0.0,0.0,0.0,75.0,803.0,1,2,1,4,1,1,53
1999-01-01 03:00:00,5199.31500,15.13,9.151200e+08,39600.0,-33.86882,151.209295,21.97,23.61,21.77,22.04,1018.0,0.0,0.0,94.0,3.60,120.0,0.0,0.0,0.0,0.0,75.0,803.0,1,3,1,4,1,1,53
1999-01-01 04:00:00,5008.23667,14.19,9.151236e+08,39600.0,-33.86882,151.209295,22.18,23.49,21.77,22.40,1018.0,0.0,0.0,89.0,3.60,120.0,0.0,0.0,0.0,0.0,75.0,803.0,1,4,1,4,1,1,53
1999-01-01 05:00:00,5022.93000,12.41,9.151272e+08,39600.0,-33.86882,151.209295,21.98,21.29,21.79,22.10,1018.0,0.0,0.0,88.0,6.20,90.0,0.0,0.0,0.0,0.0,75.0,803.0,1,5,1,4,1,1,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-16 20:00:00,8394.08000,57.47,1.600250e+09,36000.0,-33.86882,151.209295,19.75,18.03,17.22,22.22,1020.0,0.0,0.0,68.0,4.10,360.0,0.0,0.0,0.0,0.0,33.0,802.0,0,20,16,2,260,9,38
2020-09-16 21:00:00,7938.99000,55.13,1.600254e+09,36000.0,-33.86882,151.209295,18.21,14.92,15.00,21.00,1020.0,0.0,0.0,52.0,4.10,350.0,0.0,0.0,0.0,0.0,0.0,800.0,0,21,16,2,260,9,38
2020-09-16 22:00:00,7541.11000,38.92,1.600258e+09,36000.0,-33.86882,151.209295,17.11,16.53,13.89,19.86,1020.0,0.0,0.0,72.0,1.72,282.0,0.0,0.0,0.0,0.0,0.0,800.0,0,22,16,2,260,9,38
2020-09-16 23:00:00,7370.46000,38.08,1.600261e+09,36000.0,-33.86882,151.209295,16.08,15.45,14.00,18.33,1020.0,0.0,0.0,76.0,1.72,282.0,0.0,0.0,0.0,0.0,0.0,800.0,0,23,16,2,260,9,38


In [11]:
# fig = plt.figure(figsize=(18, 14))
# corr = df_example.corr()
# c = plt.pcolor(corr)
# plt.yticks(np.arange(0.5, len(corr.index), 1), corr.index)
# plt.xticks(np.arange(0.5, len(corr.columns), 1), corr.columns)
# fig.colorbar(c)

In [12]:
# sns.pairplot(df_example, vars=['consumption','max_temp','min_temp','avg', 'isHoliday','day'	,'dayofweek','dayofyear','month','weekofyear'])

In [13]:
# scaler = MinMaxScaler()
# scaler.fit(df_example)
# df_example = df_example.astype(int)
# matrixTransform=scaler.transform(df_example)
# matrixTransform[0:3]

In [14]:
# X = [item[1:10] for item in matrixTransform]
# Y = [item[0] for item in matrixTransform]

In [15]:
# df_example = df_example.astype(int)
X = np.array(df_example[['temp','temp_min','temp_max','humidity','wind_speed','wind_deg','rain_1h','snow_1h','clouds_all','isHoliday','hour','day','dayofweek','dayofyear','month','weekofyear']])
Y = np.array(df_example[['TOTALDEMAND']])

In [16]:
trnX,tesX, trnY,tesY = train_test_split(X,Y, train_size=0.8, random_state=111, shuffle=True)
# trnX,valX,trnY,valY = train_test_split(trnX,trnY, train_size=0.8, random_state=111, shuffle=True)

In [17]:
def evalutionMatrix(y_test,y_pred):
    print('Mean Squared Error  : ' + str(mean_squared_error(tesY, y_pred)))
    print('RMean Squared Error : ' + str(np.sqrt(mean_squared_error(tesY, y_pred))))
    print('Mean Absolute Error : ' + str(mean_absolute_error(y_test,y_pred)))

In [34]:
from sklearn.dummy import DummyRegressor
clf = DummyRegressor(strategy= 'mean').fit(trnX,trnY)
y_pred = clf.predict(tesX)
evalutionMatrix(tesY, y_pred)

Mean Squared Error  : 1860181.6607526904
RMean Squared Error : 1363.8847681357433
Mean Absolute Error : 1099.0092576095376


In [35]:
regr = svm.SVR()
regr.fit(trnX,trnY)
y_pred = regr.predict(tesX)
evalutionMatrix(tesY, y_pred)

Mean Squared Error  : 1671330.424672026
RMean Squared Error : 1292.7994526112802
Mean Absolute Error : 1038.40096567096


In [19]:
from sklearn.ensemble import RandomForestRegressor

regr1 = RandomForestRegressor()
regr1.fit(trnX,trnY)
y_pred = regr1.predict(tesX)
evalutionMatrix(tesY, y_pred)

Mean Squared Error  : 182383.69806711018
RMean Squared Error : 427.06404445599276
Mean Absolute Error : 331.03993219295154


In [20]:
from sklearn.linear_model import LinearRegression

regr1 = LinearRegression()
regr1.fit(trnX,trnY)
y_pred = regr1.predict(tesX)
evalutionMatrix(tesY, y_pred)

Mean Squared Error  : 1228498.8952371858
RMean Squared Error : 1108.3766937450398
Mean Absolute Error : 897.486348085512


In [21]:
from sklearn.neural_network import MLPRegressor

regr1 = MLPRegressor()
regr1.fit(trnX,trnY)
y_pred = regr1.predict(tesX)
evalutionMatrix(tesY, y_pred)

Mean Squared Error  : 696558.4106891234
RMean Squared Error : 834.6007492742403
Mean Absolute Error : 668.5426015986969


# GridSearch Implimentation

In [ ]:
# define evaluation
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

#SVR
model = svm.LinearSVR()

# define search space
space = dict()
#space['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']
# space['degree'] = [3,5]
# space['gamma'] = ['scale','auto']
# space['coef0'] = [0.0]
# space['tol'] = [0.001]
space['C'] = [1,50,100,500,1000]
space['epsilon'] = [0.1]
space['shrinking'] = [True]
space['cache_size'] = [200]
space['verbose'] = [False]
space['max_iter'] = [-1]

# define search
search = RandomizedSearchCV(model, space, n_iter=500, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv, random_state=1)

result = search.fit(X,Y)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#RandomForestRegressor
model = RandomForestRegressor()

# define search space
space = dict()
space['n_estimators'] = [10,50,100,200]
space['criterion'] = ['mse','mae']
space['random_state'] = [173]
#space['coef0'] = [0.0]
#space['tol'] = [0.001]
#space['C'] = [1,50,100,500,1000]
#space['epsilon'] = [0.1]
# space['shrinking'] = [True]
# space['cache_size'] = [200]
# space['verbose'] = [False]
# space['max_iter'] = [-1]

# define search
search = RandomizedSearchCV(model, space, n_iter=500, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv, random_state=1)

result = search.fit(X,Y)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)